In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from pathlib import Path

In [3]:
from surname.dataset import ProjectDataset
from surname.args import args

In [4]:
path = Path('../data/surnames')
scratch = path/'scratch'
surnames_csv = path/args.proc_dataset_csv
vectorizer_path = scratch/args.vectorizer_fname
args.save_dir = scratch/args.dir_name

args

Namespace(batch_size=64, checkpointer_name='classifier', checkpointer_prefix='surname', device='cuda:3', dir_name='models', early_stopping_criteria=5, learning_rate=0.001, num_epochs=100, proc_dataset_csv='surnames_with_splits.csv', raw_dataset_csv='surnames.csv', save_dir=PosixPath('../data/surnames/scratch/models'), save_every=2, save_total=5, train_proportion=0.7, vectorizer_fname='vectorizer.json')

In [5]:
df = pd.read_csv(surnames_csv)
df.head()

,nationality,nationality_index,split,surname
0,Arabic,15,train,Totah
1,Arabic,15,train,Abboud
2,Arabic,15,train,Fakhoury
3,Arabic,15,train,Srour
4,Arabic,15,train,Sayegh


In [6]:
is_load = True

In [7]:
if not is_load:
  train_ds = ProjectDataset.load_data_and_create_vectorizer(df.loc[df['split'] == 'train'])
  train_ds.save_vectorizer(vectorizer_path)

In [8]:
train_df = df.loc[df['split'] == 'train']
train_ds = ProjectDataset.load_data_and_vectorizer(train_df, vectorizer_path)
vectorizer = train_ds.get_vectorizer()
train_dl = DataLoader(train_ds, args.batch_size, shuffle=True, drop_last=True)